# Gridworld with First Visit

In [1]:
# Find the value function of policy
import numpy as np

# display output
from random import uniform
import random
import time
from IPython.display import display, clear_output

In [2]:
actions = [[-1, 0], [0, 1], [1, 0], [0, -1]] #up, right, down, left = (clockwise from up) 
action_count = len(actions) # total number of actions
gridSize = 5 # create a square grid of gridSize by gridSize
state_count = gridSize*gridSize # total number of states

In [3]:
class Gridworld():
    def __init__(self, gridSize):
        self.valueMap = np.zeros((gridSize, gridSize))
        self.states = [[i, j] for i in range(gridSize) for j in range(gridSize)]
        self.size = gridSize
        self.new_pos = [0, 0] # initialize new position for p_transition
        self.transition_prob = 1 # deterministic
    
    def initial_state(self):        # return initial state
        return grid.states[gridSize*gridSize-1]
   
    def transition_reward(self, current_pos, action): # return the transition probability

        # get next position: state: [0, 0], action: [0, 1], new_state = [0, 1]
        self.new_pos = np.array(current_pos) + np.array(action)

        # normally, reward = 0
        reward = 0

        # if new pos results in off the grid, return reward -1
        if -1 in self.new_pos or self.size in self.new_pos:
            reward = -1
        # if in state A, receive + 10
        if current_pos == [0, 1]:
            reward = 10
        # if in state B, receive + 5
        if current_pos == [0, 3]:
            reward = 5

        # if taking an action crosses the border; agent's new_pos is the same as the current pos
        if -1 in self.new_pos or self.size in self.new_pos: 
            self.new_pos = current_pos
            
        # if in state A, transition to state A'
        if current_pos == [0, 1]:
            self.new_pos = [4, 1]
            
        # if in state B, transition to state B'
        if current_pos == [0, 3]:
            self.new_pos = [2, 3]

        return self.new_pos, reward

## First-visit MC Control 

In [4]:
# create a grid object
grid = Gridworld(5)

In [5]:
# Initiate a random policy
random_policy = np.random.randint(1000, size=(state_count, action_count))
random_policy = random_policy/random_policy.sum(axis=1)[:,None]
policy = random_policy

In [6]:
# random policy
policy

array([[4.07134548e-02, 3.38503296e-01, 3.14075223e-01, 3.06708026e-01],
       [2.68200349e-01, 2.45486313e-01, 2.09085614e-01, 2.77227723e-01],
       [1.93441537e-01, 3.06392845e-01, 3.20635972e-01, 1.79529646e-01],
       [3.56673961e-01, 2.64223195e-01, 6.56455142e-03, 3.72538293e-01],
       [3.37572674e-01, 1.70784884e-01, 1.66787791e-01, 3.24854651e-01],
       [3.22834646e-01, 2.35193427e-01, 1.57480315e-01, 2.84491612e-01],
       [7.74299835e-02, 7.86655684e-01, 5.43657331e-02, 8.15485997e-02],
       [2.16335541e-01, 2.62693157e-01, 2.68579838e-01, 2.52391464e-01],
       [3.97546570e-01, 3.61199455e-01, 2.31712858e-02, 2.18082690e-01],
       [2.51314406e-01, 7.36067298e-02, 3.92744479e-01, 2.82334385e-01],
       [2.83745440e-01, 3.26712606e-01, 2.52533441e-01, 1.37008512e-01],
       [4.18223235e-01, 1.28473804e-01, 3.43963554e-01, 1.09339408e-01],
       [1.97752809e-01, 3.49812734e-01, 8.61423221e-02, 3.66292135e-01],
       [3.68303571e-01, 5.01488095e-01, 3.57142857e

### Create an Episode following Policy

In [7]:
def generate_episode(steps):

    # set initial state
    state = grid.initial_state()

    # initialize state (with iniitial state), action list and reward list
    state_list = [state]
    action_list = []
    reward_list = []

    # generate an episode
    for i in range(steps):

        # pick an action based on categorical distribution in policy
        action = int(np.random.choice(action_count, 1, p=policy[grid.states.index(state)])) # get index in int
        action = actions[action] # convert the integer index (ie. 0) to action (ie. [-1, 0])

        # get new state and reward after taking action from current state
        new_state, reward = grid.transition_reward(state, action)
        state = list(new_state)

        # save state, action chosen and reward to list
        state_list.append(state)
        action_list.append(action)
        reward_list.append(reward)
        
    return state_list, action_list, reward_list

### First Visit MC

In [8]:
# initialize q values for all state action pairs
Q_values = np.zeros((state_count, action_count))

In [9]:
# intialize parameters
gamma = 0.99
epsilon = 0.2

In [10]:
# define average function
def Average(lst): 
    return sum(lst) / len(lst) 

In [11]:
# iterate 500 times: each time, generating an episode of 200 steps
max_steps = 200

# define variables for keeping track of time steps
Terminal = max_steps
t_list=[]
for i in range(1,max_steps+1):
    t = Terminal - i
    t_list.append(t)

In [12]:
# iteration 500 times
for iteration in range(500):
  
    # generate an episode of specified step count
    state_list, action_list, reward_list = generate_episode(max_steps)
    
#     print("state_list: ", state_list)
#     print("action_list: ", action_list)
#     print("reward_list: ", reward_list)
    
    # intialize G
    G = 0
    
    # initiate returns and visited list to none
    returns_list = []
    visited_list = []

    # loop for each step of episode: T-1, T-2, T-3 ... 0 = 199, 198, 197 ... 0
    for t in t_list:

        # calculate G: starting with the last reward at index t (naturally accounts for pseudocode's "t-1")
        G = gamma*G + reward_list[t]
        # print(G)
        
        # combine state action pair, for example, state = [0,0], action = [0,1], state_action_pair = [0,0,0,1]
        state_action_pair = []
        state_action_pair.extend(state_list[t])
        state_action_pair.extend(action_list[t])
        # print(state_action_pair)

        # check if state action pair have been visited before (if not: continue, else: move to the next time step)
        if state_action_pair not in visited_list:

            # add state action pair to visited list
            visited_list.append(state_action_pair)

            # append G to returns
            returns_list.append(G)

            # find state and action index, for example, converting action [-1, 0] to 0, and same for state #
            state_index = grid.states.index(state_list[t])
            action_index = actions.index(action_list[t])
    #         print("state_index: ", state_index)

            # write Q_values to the state-action pair
            Q_values[state_index][action_index] = Average(returns_list)
    #         print("average return: ", Average(returns_list))

            # choose best action at given state
            choose_action = np.argmax(Q_values[state_index])

            # overwrite policy
            for a in range(action_count): # for action in actions [0, 1, 2, 3]
                if choose_action == a: # if the choose_action is the same as the current action
                    policy[state_index][a] = 1 - epsilon + epsilon/action_count 
                else: # if choose_action is not the same as the current action
                    policy[state_index][a] = epsilon/action_count

In [13]:
# total unique state action pairs at the end of one episode
print(len(visited_list))

21


In [14]:
Q_values

array([[ -1.        , -21.93728173, -23.01381927, -23.64873295],
       [-14.60457783, -14.08476552, -21.2935195 , -23.93988266],
       [ -3.832771  ,  -0.32834918,   2.64653417,   3.38598   ],
       [ -0.95621594, -16.48602462,  -3.49026927, -23.98747016],
       [ -4.06834033,  -3.49119409, -22.33438353, -18.70148161],
       [ -1.48505   ,  -0.28338226, -14.8900471 , -22.52166945],
       [ -0.72989558,   0.42648852,   0.        ,   0.        ],
       [ -0.12607869,  -1.24944479,   0.        ,   0.        ],
       [  0.71071429,  -7.64088088,  -9.43686549,   0.        ],
       [-20.56809691,  -6.15909425,  -4.11254652, -13.82617144],
       [ -1.64016633, -26.77957433, -14.37776955,  -1.9628485 ],
       [  0.04444855,  -0.26622266,  -1.08708751,  -7.00986816],
       [ -1.30870379,   0.        ,   0.        ,  -1.28020037],
       [ -8.67780017,  -5.23335485,  -8.10087216,   0.        ],
       [-11.49931407,  -1.        ,  -3.10494762,  -9.99463502],
       [  0.88304961, -12

In [15]:
#up, right, down, left = (clockwise from up) 
policy

array([[0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.05, 0.85, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.05, 0.05, 0.05, 0.85]])

## Visualize 

In [16]:
# PRINT POLICY TABLE ################################################################################
# import pandas library
import pandas as pd
# define column and index
columns=range(grid.size)
index = range(grid.size)
# define dataframe to represent policy table
policy_table = pd.DataFrame(index = index, columns=columns)

# iterate through policy to make a table that represents action number
# as action name (eg. left, right, up, down)
for state in range(len(policy)):
    
    # find the best action at each state
    best_action = np.argmax(policy[state])

    # get action name
    if best_action == 0:
        action_name = 'up'
    elif best_action == 1:
        action_name = 'right'
    elif best_action == 2:
        action_name = 'down'
    else:
        action_name = 'left'

    # calculate the row and column coordinate of the current state number
    row = int(state/grid.size)
    column = round((state/grid.size - int(state/grid.size))*grid.size)
            
    # assign action name
    policy_table.loc[row][column] = action_name

print("Policy Table: ")
print(policy_table)
print()

Policy Table: 
       0      1      2     3      4
0     up  right   left    up  right
1  right  right   down    up   down
2     up     up  right  left  right
3     up  right     up    up   down
4     up  right     up  left   left

